# 第 2 课：Python 进阶语法与常用技巧

## 本课目标
- 掌握函数定义与参数形式，理解作用域与命名空间
- 学会模块导入、异常处理、文件读写与上下文管理
- 理解生成器、迭代器、匿名函数与高阶函数的常见用法
- 了解面向对象的基本写法与可变/不可变类型的参数传递行为
- 能用基本调试/测试方法定位问题，并了解虚拟环境和依赖管理的入门做法

> 建议：每个代码单元先运行，再修改参数观察行为。

## 1. 函数（`def`）与参数形式

什么是函数：函数是把可以重复使用的一段代码‘打包’起来，给它一个名字，后面通过名字调用。

基本语法：
- 使用 `def 函数名(参数):` 定义，函数体通过缩进书写，`return` 返回值（可以没有返回，默认为 `None`）。

参数类型（常见）解释：
- 位置参数：按顺序传入的参数。
- 关键字参数：传入时使用 `name=value` 的形式，顺序可变。
- 默认参数：在定义时给出默认值，如果调用时省略则使用默认值。
- 可变参数 `*args`：接受多个位置参数，内部作为元组使用。
- 可变关键字 `**kwargs`：接受多个关键字参数，内部作为字典使用。

小贴士：给函数写文档字符串（docstring）能帮助别人或未来的你快速理解函数用途。


In [6]:
def greet(name, greeting='Hello', *args, sep=' ', **kwargs):
    """
    参数说明：
    - name (str): 必需，表示被称呼的人名。
    - greeting (str): 可选，问候语，默认 'Hello'。
    - *args (tuple): 可选，接收额外的位置参数，函数内部作为元组处理。
    - sep (str): 可选关键字参数（关键字-only），用于连接输出部分，默认空格。
    - **kwargs (dict): 可选，接收额外的关键字参数，函数内部作为字典处理。
    返回值：
    - 返回一个字符串，由上述各部分按 sep 连接而成。
    """
    # 把可变位置参数转成字符串的一段示例处理逻辑
    extra = ','.join(map(str, args)) if args else ''
    # 从 kwargs 里取一个名为 'info' 的附加信息（如果存在）
    info = kwargs.get('info', '')
    parts = [greeting, name]
    if extra:
        parts.append(extra)
    if info:
        parts.append(info)
    return sep.join(parts)

# 示例调用和对应映射（下列调用演示了实参如何对应到形参）
print("调用1: greet('Alice') ->", greet('Alice'))  # name='Alice'
print("调用2: greet('Bob', 'Hi') ->", greet('Bob', 'Hi'))  # name='Bob', greeting='Hi'
print("调用3: greet('Cathy', 1, 2, info='来自 kwargs') ->", greet('Cathy', 'Hi', 1, 2, info='来自 kwargs'))
# 解释：在上面这行里，'Cathy' 对应 name；1 和 2 被收集到 *args（即元组）；info='来自 kwargs' 被收集到 **kwargs（即字典）
print("调用4: greet(name='Dan', greeting='Hey', sep=' - ') ->", greet(name='Dan', greeting='Hey', sep=' - '))  # 使用关键字参数调用

# 如果想查看调用时的位置参数和关键字参数分别是什么，可以这样写一个小工具函数来打印映射：
def show_call_mapping(*call_args, **call_kwargs):
    print('--- call mapping ---')
    print('位置参数 tuple ->', call_args)
    print('关键字参数 dict ->', call_kwargs)

show_call_mapping('Cathy', 1, 2, info='来自 kwargs')
show_call_mapping(name='Dan', greeting='Hey', sep=' - ')

调用1: greet('Alice') -> Hello Alice
调用2: greet('Bob', 'Hi') -> Hi Bob
调用3: greet('Cathy', 1, 2, info='来自 kwargs') -> Hi Cathy 1,2 来自 kwargs
调用4: greet(name='Dan', greeting='Hey', sep=' - ') -> Hey - Dan
--- call mapping ---
位置参数 tuple -> ('Cathy', 1, 2)
关键字参数 dict -> {'info': '来自 kwargs'}
--- call mapping ---
位置参数 tuple -> ()
关键字参数 dict -> {'name': 'Dan', 'greeting': 'Hey', 'sep': ' - '}


### 参数对应关系（简单说明）

- 当你写函数调用时，按位置给出的实参会依次对应到函数定义里的位置参数（从左到右）。
- 如果函数定义里有 `*args`，所有多余的按位置传入的实参会被打包成一个元组，赋给 `args`。
- 在调用时以 `name=value` 形式给出的实参会被收集到 `**kwargs`（字典）。
- 出于可读性，推荐对重要参数使用关键字参数调用（例如 `greet(name='Tom', greeting='Hi')`）。

示例回顾：`greet('Cathy', 1, 2, info='来自 kwargs')` 中：
- `'Cathy'` 对应 `name`（通常为字符串）；
- `1, 2` 被收集到 `args`（函数内部作为元组），此处示例把它们转为字符串再拼接；
- `info='来自 kwargs'` 被收集到 `kwargs`（字典），可以通过 `kwargs['info']` 或 `kwargs.get('info')` 读取。


## 2. 作用域与命名空间（LEGB）

为什么需要作用域：作用域决定了在某个位置名字（变量/函数名）如何被查找和绑定，避免名字冲突。

LEGB 规则（查找顺序）：
- Local：当前函数内部的名字。
- Enclosing：外层（嵌套）函数的名字（仅用于嵌套函数）。
- Global：模块级别的名字（文件中定义的全局变量）。
- Builtins：Python 内置名字（如 `len`, `print`）。

修改作用域：
- `global`：在函数中声明某名为全局变量，函数内的赋值将修改全局变量。
- `nonlocal`：用于嵌套函数中，声明要修改外层（不是全局）作用域的变量。

例子会在下方代码单元展示（运行并观察输出）。


In [ ]:
x = 'global_x'

def outer():
    x = 'enclosing_x'
    def inner():
        nonlocal x
        x = 'modified_in_inner'
        print('inner x =', x)
    inner()
    print('outer x =', x)

outer()
print('global x =', x)

## 3. 模块与包：`import` 使用与 `__name__ == '__main__'`

模块是什么：一个 .py 文件就是一个模块，模块可以把相关函数/类组织在一起，方便复用。包是带有 `__init__.py` 的文件夹（现代 Python 可选），用于组织多个模块。

常见导入方式：
- `import math`：导入模块，使用时写 `math.sqrt()`。
- `from math import sqrt`：直接导入模块中的一个名字，可直接调用 `sqrt()`。
- `import package.module as mod`：使用别名减少书写。

关于 `__name__ == '__main__'`：当一个模块被直接运行（而不是被导入）时，`__name__` 等于 `'__main__'`，常把测试或演示代码放在这段判断下，保证被导入时不会执行。

提示：把可重复使用的函数写进模块，然后在其他文件中导入，这样项目更清晰。


In [1]:
# 示例：标准库导入与别名
import math
from collections import defaultdict

print('sqrt(16)=', math.sqrt(16))
d = defaultdict(int)
d['a'] += 1
print(d)

# 如果这个文件被当作脚本执行，就运行下面的测试代码
if __name__ == '__main__':
    print('作为脚本运行时会看到这行')

sqrt(16)= 4.0
defaultdict(<class 'int'>, {'a': 1})
作为脚本运行时会看到这行


## 4. 异常处理（`try/except/finally`）与自定义异常

为什么需要异常处理：在运行中可能出现错误（例如除以 0、文件不存在等），使用异常处理可以优雅地捕获错误、提示用户或做清理工作，而不是直接崩溃。

基本结构：
```
try:
    # 可能抛出异常的代码
except SomeError as e:
    # 处理特定异常
else:
    # 没有异常时执行
finally:
    # 无论是否发生异常都执行（常用于释放资源）
```

自定义异常：通过继承 `Exception` 创建，便于区分和处理特定错误类型。

示例在下方代码单元演示异常链与 finally。


In [ ]:
class MyError(Exception):
    pass

def div(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise MyError('division by zero') from e
    finally:
        print('finished div')

try:
    div(1, 0)
except MyError as e:
    print('捕获到自定义异常:', e)

## 5. 文件 I/O 与上下文管理（`with`）

文件读写基础：
- 打开文件使用 `open(path, mode)`，模式常见有 `'r'`（读）、`'w'`（写，覆盖）、`'a'`（追加）、`'b'`（二进制）。
- 使用 `with open(...) as f` 可以确保文件使用完后自动关闭，避免资源泄露。

推荐使用 `pathlib.Path` 来构造和处理路径，语义更清晰且跨平台。

下面的代码示例会在当前目录创建一个小文件并读取它，运行后查看输出。


In [7]:
from pathlib import Path
p = Path('sample.txt')

# 写文件（自动关闭）
with p.open('w', encoding='utf-8') as f:
    f.write('第一行\n第二行\n')

# 读文件：逐行读取并去掉末尾换行符
with p.open('r', encoding='utf-8') as f:
    for line in f:
        print('line->', line.strip())

line-> 第一行
line-> 第二行


## 6. 字符串格式化与常用方法

常见需求：把变量插入一段可读文字，或按分隔符拆分/合并字符串。

常用写法：
- f-string（推荐）：`f'{name} is {age}'`，简洁且可读。
- `str.format()`：`'{0} is {1}'.format(name, age)`，兼容旧版本。
- 常用方法：`split()`（拆分）、`join()`（合并）、`strip()`（去除两端空白）、`replace()`（替换）。

示例在下方代码单元，运行并尝试修改变量内容观察变化。


In [8]:
name = 'Alice'
age = 30
print(f'{name} is {age} years old')
print('{0} is {1} years'.format(name, age))
s = ' a,b,c '
print(s.strip())
print(s.split(','))
print('-'.join(['a','b','c']))

Alice is 30 years old
Alice is 30 years
a,b,c
[' a', 'b', 'c ']
a-b-c


## 7. 常用内建函数与类型转换

Python 提供许多内建函数来简化常见操作：
- 类型转换：`int()` / `float()` / `str()` / `bool()`。
- 聚合函数：`len()`、`sum()`、`min()`、`max()`。
- 排序和转换：`sorted()`、`map()`（对每个元素应用函数）、`filter()`（筛选满足条件的元素）。

提示：`map` 和 `filter` 返回迭代器（在 Python3 中），要得到列表请用 `list()` 包装。


In [9]:
nums = ['3', '1', '2']
nums_int = list(map(int, nums))
print(sorted(nums_int))
print(sum(nums_int), min(nums_int), max(nums_int))
print(any([0, '', None, 5]))
print(all([1, 2, 3]))

[1, 2, 3]
6 1 3
True
True


解释：`nums_int = list(map(int, nums))` 的含义与等价写法：

- `map(int, nums)` 会对 `nums` 序列中的每个元素应用 `int` 函数，返回一个 `map` 对象（在 Python 3 中是惰性迭代器）。
- 使用 `list(...)` 将该迭代器转换为列表，从而得到每个字符串元素转换后的整数列表。

常见等价写法（行为相同）：
- 列表推导式：`nums_int = [int(x) for x in nums]`
- 使用显式循环：
  nums_int = []
  for x in nums:
      nums_int.append(int(x))

何时使用：当你需要把表示数字的字符串列表转换为数字类型（如整型）以便进行数值计算（例如求和、排序）时使用。

注意：如果 `nums` 中包含不能转换为整数的字符串（例如 'a'），调用 `int()` 会抛出 `ValueError`，可用 `try/except` 或先过滤数据来处理异常情况。

## 8. 集合（`set`）与集合运算

集合是一个无序、不重复的容器，适合用于去重或做集合运算（并集/交集/差集）。

常用操作：
- 去重：`set(list)`。
- 并集：`a | b` 或 `a.union(b)`。
- 交集：`a & b` 或 `a.intersection(b)`。
- 差集：`a - b` 或 `a.difference(b)`。

注意：集合内元素必须是可哈希的（如数字、字符串、元组），不能放列表或字典。


In [ ]:
a = {1,2,3,3}
b = {3,4}
print('a=', a)
print('a & b=', a & b)
print('a | b=', a | b)
print('a - b=', a - b)

## 9. 生成器与迭代器（`yield`）

生成器的作用：用于惰性生成序列（按需计算），避免一次性把所有数据放入内存。适合处理大数据或流式数据。

创建方式：
- 生成器函数：包含 `yield` 的函数，每次 `yield` 返回一个值，函数会在下一次 `next()` 时从上次位置继续。
- 生成器表达式：类似于列表推导，但用圆括号，如 `(x*x for x in range(10))`。

示例与常见场景在下方代码单元。


In [11]:
def gen_squares(n):
    for i in range(n):
        yield i*i

g = gen_squares(5)
print(next(g))
print(list(gen_squares(6)))
# 生成器表达式
g2 = (i*i for i in range(5))
print(next(g2))
print(list(g2))

0
[0, 1, 4, 9, 16, 25]
0
[1, 4, 9, 16]


## 10. 匿名函数 `lambda` 与高阶函数

`lambda` 表达式用于定义很短的、一次性的函数，常在需要函数作为参数时临时使用。

高阶函数示例：
- `map(func, iterable)`：对每个元素应用 `func`，返回迭代器。
- `filter(func, iterable)`：筛选出使 `func` 返回 True 的元素，返回迭代器。
- 在 `sorted()` 中传入 `key=` 参数来定制排序规则。

提示：当逻辑变复杂时，最好定义带名字的普通函数，而不是在 `lambda` 中写复杂表达式。


In [12]:
nums = [1,2,3,4]
print(list(map(lambda x: x*2, nums)))
print(list(filter(lambda x: x%2==0, nums)))
print(sorted(['b','aa','c'], key=lambda s: len(s)))

[2, 4, 6, 8]
[2, 4]
['b', 'c', 'aa']


## 11. 面向对象入门（`class`）

为什么用类：当你需要把数据和对数据的操作封装在一起，并创建多个行为一致的对象时，类很有用。

基础知识点：
- `class` 定义类，`__init__` 用于初始化实例属性。
- 实例方法：第一个参数通常是 `self`，表示实例自身。
- 类属性 vs 实例属性：类属性由所有实例共享，实例属性只属于该实例。
- 继承：子类可以扩展或重写父类的方法。

示例代码会演示一个简单的继承与方法重写。


In [ ]:
class Animal():
    # 父类：动物
    species = 'Unknown'
    def __init__(self, name):
        self.name = name
    def speak(self):
        return f'{self.name} makes a sound'

class Dog(Animal):
    # 子类：动物里的狗，会继承父类的属性和方法，也可重新定义，如下方重新定义speak方法
    species = 'Dog'
    def speak(self):
        return f'{self.name} barks'

d = Dog('Rex') # Rex是狗的名字,是类init的参数name
print(d.species, d.speak())

Dog Rex barks


## 12. 可变性与参数传递（重要陷阱）

Python 参数传递的关键点：传递的是对象的引用（传引用），但表现为“按值传递引用”。这意味着：
- 可变对象（例如列表、字典）在函数内部被修改，会影响到外部的对象。
- 不可变对象（例如整数、字符串、元组）在函数内部修改会创建新的对象，外部不受影响。

避免常见陷阱的做法：不要在函数定义中使用可变对象作为默认参数；如果需要，使用 `None` 并在函数内部创建新对象。

下方代码单元演示了这一点并给出安全的写法。


In [ ]:
def append_item(lst):
    lst.append('X')

a = [1,2]
append_item(a)
print(a)  # a 被修改了

def safe_append(lst=None):
    if lst is None:
        lst = []
    lst.append('Y')
    return lst

print(safe_append())
print(safe_append())  # 重新生成了一个lst，因此两次结果相同，不会在之前的列表上追加

[1, 2, 'X']
['Y']
['Y']


## 13. 调试与测试基础

调试入门建议：
- 在不知道问题在哪时，可先用 `print()` 打印关键变量（快速且直观）。
- 复杂问题使用调试器：在命令行或 IDE 中使用断点（例如 VS Code 的断点功能），可以逐步执行并查看变量。
- 写测试用例能防止回归，常用`pytest` 但需额外安装，一般断点debug足够了。


## 14. 环境与依赖管理、代码风格

环境管理：
- 使用虚拟环境（`python -m venv venv`）为每个项目隔离依赖，避免版本冲突。或用管理虚拟环境的软件如anaconda，为不同项目创建不同依赖环境，导入不同包。
- 一般开源项目会配置requirements.txt，使用 `pip install -r requirements.txt` 来批量安装依赖，对于单独的依赖也常用pip install工具安装，示例：`pip install numpy`安装numpy库。

代码风格建议（编程规范）：
- 使用有意义的变量名，函数名用小写加下划线（snake_case），类名用驼峰命名法（CamelCase）。
- 缩进 4 个空格，不要混用 Tab 和空格。
- 每行不宜过长（一般 <80/100 字符），多用函数拆分复杂逻辑。

遵守这些可使代码更容易被他人（或未来的你）阅读。
